In [ ]:
import requests
import json

class VulnerabilityScanner:
    def __init__(self):
        # CVE API endpoints
        self.cve_url = 'https://cve.circl.lu/api/cve/'
        self.cpe_url = 'https://cve.circl.lu/api/cpe2.2/{}'
        self.vulnerabilities = {}

    def fetch_cve_data(self, cve_id):
        try:
            response = requests.get(f'{self.cve_url}{cve_id}')
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Failed to fetch CVE data for {cve_id}: {e}")
            return None

    def fetch_cpe_matches(self, package_name):
        try:
            response = requests.get(self.cpe_url.format(package_name))
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Failed to fetch CPE data for {package_name}: {e}")
            return None

    def scan_application(self, package_list):
        for package, version in package_list.items():
            print(f"Scanning {package} version {version}...")
            cpe_data = self.fetch_cpe_matches(package)
            if cpe_data:
                vulnerabilities = self.check_vulnerabilities(cpe_data, version)
                self.vulnerabilities[package] = vulnerabilities
            else:
                print(f"No CPE data found for package: {package}")
        return self.vulnerabilities

    def check_vulnerabilities(self, cpe_data, version):
        package_vulnerabilities = []
        for cpe_entry in cpe_data:
            cpe_name = cpe_entry.get('title', 'Unknown')
            cve_list = cpe_entry.get('vulnerable_configuration', [])
            for cve in cve_list:
                cve_data = self.fetch_cve_data(cve)
                if cve_data and version in cve_data.get('vulnerable_configuration', []):
                    package_vulnerabilities.append({
                        'cve_id': cve,
                        'summary': cve_data.get('summary', 'No description available'),
                        'severity': cve_data.get('cvss', {}).get('score', 'N/A'),
                        'references': cve_data.get('references', [])
                    })
        return package_vulnerabilities

    def generate_report(self):
        report = {"vulnerabilities": self.vulnerabilities}
        with open('vulnerability_report.json', 'w') as f:
            json.dump(report, f, indent=4)
        print("Vulnerability report generated: vulnerability_report.json")

# Usage example
scanner = VulnerabilityScanner()
package_list = {'openssl': '1.0.2', 'nginx': '1.16.0'}
vulnerabilities = scanner.scan_application(package_list)
scanner.generate_report()